# LogFormers and reformers

In [ ]:
## Transformers library from hugging face.
!pip install transformers

     |████████████████████████████████| 1.3MB 8.8MB/s 
     |████████████████████████████████| 890kB 54.1MB/s 
     |████████████████████████████████| 1.1MB 43.4MB/s 
     |████████████████████████████████| 2.9MB 45.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=f42eb969304ed34559eb21f6f682c81513040b9ed3e77b356d35e7e14939b8b5
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from transformers import LongformerTokenizer, LongformerForQuestionAnswering
import torch

In [ ]:
tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-large-4096-finetuned-triviaqa")
model = LongformerForQuestionAnswering.from_pretrained("allenai/longformer-large-4096-finetuned-triviaqa", return_dict=True)

In [ ]:
question, text = "Who is karteek Menda?", "Karteek Menda is the name of the person"
encoding = tokenizer(question, text, return_tensors="pt")
input_ids = encoding["input_ids"]

# default is local attention everywhere
# the forward method will automatically set global attention on question tokens
attention_mask = encoding["attention_mask"]

outputs = model(input_ids, attention_mask=attention_mask)
start_logits = outputs.start_logits
end_logits = outputs.end_logits
all_tokens = tokenizer.convert_ids_to_tokens(input_ids[0].tolist())

answer_tokens = all_tokens[torch.argmax(start_logits) :torch.argmax(end_logits)+1]
# remove space at the beginning space token
answer = tokenizer.decode(tokenizer.convert_tokens_to_ids(answer_tokens)) 

/usr/local/lib/python3.6/dist-packages/transformers/modeling_longformer.py:71: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  sep_token_indices = (input_ids == sep_token_id).nonzero()


In [ ]:
print(start_logits)
print(end_logits)

tensor([[-12.7956, -12.5852, -13.2697, -11.8696, -13.7572, -14.0177, -12.5711,
         -14.3006, -12.5679,  -9.9897, -11.8166,  -6.2169, -13.1365, -12.8815,
          -9.0413, -12.5384, -12.2443,  -9.8821,  -9.5378,  -8.5282,  -9.5897,
          -3.8854,  -8.1345]], grad_fn=<SqueezeBackward1>)
tensor([[-13.7251, -13.1181, -13.3138, -14.5676, -13.2124, -12.3004, -13.8996,
         -11.5341, -13.3891, -12.6155, -12.2707, -12.2510, -12.0367,  -7.8594,
         -13.0387,  -5.4692, -12.2018, -11.9587, -10.3666, -11.7762, -11.8965,
          -3.2758,  -7.8142]], grad_fn=<SqueezeBackward1>)


In [ ]:
print(answer)

 person


# helper function

In [ ]:
def get_answer(question, context):
  # encode question and context so that they are seperated by a tokenizer.sep_token and cut at max_length
  encoding = tokenizer.encode_plus(question, context, return_tensors="pt", max_length=4096)
  input_ids = encoding["input_ids"].to("cuda")
  attention_mask = encoding["attention_mask"].to("cuda")

  # the forward method will automatically set global attention on question tokens
  # The scores for the possible start token and end token of the answer are retrived
  # wrap the function in torch.no_grad() to save memory
  with torch.no_grad():
    start_scores, end_scores = model(input_ids=input_ids, attention_mask=attention_mask)

  # Let's take the most likely token using `argmax` and retrieve the answer
  all_tokens = tokenizer.convert_ids_to_tokens(encoding["input_ids"][0].tolist())
  answer_tokens = all_tokens[torch.argmax(start_scores): torch.argmax(end_scores)+1]
  answer = tokenizer.decode(tokenizer.convert_tokens_to_ids(answer_tokens))[1:].replace('"', '')  # remove space prepending space token and remove unnecessary '"'
        
  return answer

# reformer

In [ ]:
from transformers import ReformerTokenizer, ReformerForQuestionAnswering
import torch
tokenizer = ReformerTokenizer.from_pretrained('google/reformer-crime-and-punishment')

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [ ]:
model = ReformerForQuestionAnswering.from_pretrained('google/reformer-crime-and-punishment', return_dict=True)
question, text = "Who is karteek Menda?", "Karteek Menda is the name of the person"
inputs = tokenizer(question, text, return_tensors='pt')
start_positions = torch.tensor([1])
end_positions = torch.tensor([3])
outputs = model(**inputs, start_positions=start_positions, end_positions=end_positions)
loss = outputs.loss
start_scores = outputs.start_logits
end_scores = outputs.end_logits

Some weights of the model checkpoint at google/reformer-crime-and-punishment were not used when initializing ReformerForQuestionAnswering: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing ReformerForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing ReformerForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ReformerForQuestionAnswering were not initialized from the model checkpoint at google/reformer-crime-and-punishment and are newly initialized: ['reformer.encoder.layers.0.attention.self_attention.mask_value_float16', 'reformer.encoder.layers.0.attention.self_attention.mask_value_float32

In [ ]:
start_scores#Span-start scores (before SoftMax).

tensor([[-0.7459, -0.3572, -0.4044, -0.7454, -0.1798,  0.1178, -0.5724, -0.4533,
         -0.1996,  0.3512, -0.0284, -0.3686,  0.4681,  0.1056, -0.0812,  0.6946,
         -0.0262,  0.1570, -0.4402, -0.4041,  0.4901,  0.1441, -0.3004,  0.5577,
         -0.1372, -0.3461,  0.6401,  0.4186,  0.2149, -0.3437,  0.3191,  0.7072,
          0.7100, -0.1301]], grad_fn=<SqueezeBackward1>)

In [ ]:
end_scores#Span-end scores (before SoftMax).

tensor([[ 0.2349,  0.7090, -0.3287,  0.0874, -0.0187, -0.5926,  0.0608, -0.7348,
         -0.2905,  0.0812,  0.7109,  0.0893,  0.2206,  0.0156,  0.3720,  0.0406,
          0.1311, -0.8069, -0.8942, -0.2086,  0.4591,  0.2474, -0.4452,  0.1637,
         -0.3744, -0.5917,  0.2320, -0.0589,  0.2775, -0.6218, -0.1378, -0.0916,
         -0.2663,  0.5539]], grad_fn=<SqueezeBackward1>)

In [ ]:
loss #Total span extraction loss is the sum of a Cross-Entropy for the start and end positions.

tensor(3.7178, grad_fn=<DivBackward0>)